In [3]:
from LoadingBMWDataset import LoadingBMWDataset
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
import json
import os
from PIL import Image


In [5]:
# Load the dataset by instantiating the class LoadingBMWDataset
dataset = LoadingBMWDataset(images_dir='/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/images',
                             labels_dir='/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/json')

# Splitting the dataset into 80% training and 20% validation
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [8]:
# Function converting the bounding box coordinates from (left, top, right, bottom) to (x, y, width, height)
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box['Left'] + box['Right']) / 2.0 - 1
    y = (box['Top'] + box['Bottom']) / 2.0 - 1
    w = box['Right'] - box['Left']
    h = box['Bottom'] - box['Top']
    return x * dw, y * dh, w * dw, h * dh

In [9]:
# Function converting the annotations from json to yolo format
def convert_annotations(json_dir, image_dir, output_dir):
    class_ids = {'bin': 0, 'dolly': 1, 'jack': 2} # Example class mapping

    for json_file in os.listdir(json_dir):
        if json_file.endswith(".json"):
            with open(os.path.join(json_dir, json_file)) as f:
                data = json.load(f)
                image_path = os.path.join(image_dir, json_file.replace('.json', '.jpg'))
                image = Image.open(image_path)
                width, height = image.size
                
                output_file = json_file.replace('.json', '.txt')
                with open(os.path.join(output_dir, output_file), 'w') as out:
                    for obj in data:
                        class_id = class_ids[obj['ObjectClassName']]
                        bbox = convert_bbox((width, height), obj)
                        out.write(f"{class_id} {' '.join([str(a) for a in bbox])}\n")

In [10]:
# Implementing the functions defined, converting the annotations from json to yolo format
json_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/json'
image_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/images'

# Storing the converted annotations in the output directory
output_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/yolo'
convert_annotations(json_dir, image_dir, output_dir)